# 🔍 VerityNgn - YouTube Video Verification

Welcome to VerityNgn! This notebook allows you to verify YouTube videos using AI-powered analysis.

## What This Does

1. **Analyzes YouTube videos** using multimodal AI (video, audio, text)
2. **Extracts claims** from the video content
3. **Searches for evidence** across the web
4. **Generates truthfulness reports** with confidence scores

## Prerequisites

You need a **VerityNgn API endpoint**. This can be:
- Local: `http://localhost:8080` (running on your machine)
- Cloud Run: `https://your-service.run.app`
- Custom deployment: Any accessible URL

## Quick Start

1. Run the **Setup** section below
2. Configure your **API endpoint**
3. Enter a **YouTube URL**
4. Submit and wait for results!


## 📦 Setup

Install required packages (only needs to run once per session)


In [ ]:
# Install minimal dependencies
!pip install -q requests ipython

print("✅ Setup complete!")


## ⚙️ Configuration

Set your API endpoint URL here


In [ ]:
import requests
import time
import json
from IPython.display import HTML, display, clear_output

# ===== CONFIGURATION =====
# Change this to your API endpoint
#
# OPTIONS:
# 1. Cloud Run: "https://your-service.run.app"
# 2. ngrok Tunnel: "https://abc123.ngrok-free.app" (see NGROK_QUICKSTART.md)
# 3. Local: "http://localhost:8080" (won't work from Colab unless tunneled)
#
API_URL = "http://localhost:8080"  # ⚠️ CHANGE THIS to your API URL

# Test connection
print("🔗 Testing connection to API...")
print(f"   URL: {API_URL}")
print("")
try:
    response = requests.get(f"{API_URL}/health", timeout=5)
    if response.ok:
        print(f"✅ Connected to API successfully!")
        print(f"   Status: {response.json().get('status', 'OK')}")
    else:
        print(f"⚠️  API responded but not healthy: {response.status_code}")
except requests.exceptions.ConnectionError:
    print(f"❌ Cannot connect to API at {API_URL}")
    print("")
    print("📝 Troubleshooting:")
    print("   1. Is the API running?")
    print("      docker compose ps api")
    print("")
    print("   2. Using localhost from Colab? Use ngrok tunnel instead:")
    print("      ./scripts/start_ngrok_tunnel.sh")
    print("      Then update API_URL to your ngrok URL")
    print("")
    print("   3. Cloud deployment:")
    print("      See docs/DEPLOYMENT_CLOUD_RUN.md")
except Exception as e:
    print(f"❌ Error: {e}")


## 🎬 Submit Video for Verification

Enter a YouTube URL and submit for analysis


In [ ]:
# Enter your YouTube URL here
VIDEO_URL = "https://www.youtube.com/watch?v=dQw4w9WgXcQ"

# Optional: Configuration overrides
CONFIG = {
    "model_name": "gemini-2.0-flash-exp",
    "max_claims": 20,
    "temperature": 0.7
}

print(f"📤 Submitting: {VIDEO_URL}")
print(f"   Model: {CONFIG['model_name']}")
print(f"   Max Claims: {CONFIG['max_claims']}")
print()

try:
    response = requests.post(
        f"{API_URL}/api/v1/verification/verify",
        json={
            "video_url": VIDEO_URL,
            "config": CONFIG
        },
        timeout=30
    )
    
    response.raise_for_status()
    result = response.json()
    
    TASK_ID = result.get("task_id")
    
    if TASK_ID:
        print(f"✅ Task submitted successfully!")
        print(f"   Task ID: {TASK_ID}")
        print()
        print("Run the next cell to monitor progress...")
    else:
        print("❌ API did not return a task_id")
        print(f"   Response: {result}")
        
except requests.exceptions.HTTPError as e:
    print(f"❌ HTTP Error: {e}")
    if e.response is not None:
        try:
            error_detail = e.response.json().get('detail', str(e))
            print(f"   Detail: {error_detail}")
        except:
            print(f"   Response: {e.response.text}")
except Exception as e:
    print(f"❌ Error: {e}")
